In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikeras

import os
from PIL import Image
import numpy as np
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Definir a pasta que contém as imagens
img_dir = '/content/drive/MyDrive/DIO/fake_natty/fotos'

# Collect the images from the folder, handling potential errors
img_files = []
try:
    img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.jpg')]
    if not img_files:
        raise FileNotFoundError("No '.jpg' files found in the directory.")  # Raise an error if no images are found
except FileNotFoundError as e:
    print(f"Error: {e}")  # Print the error message
    # Handle the error appropriately, e.g., exit the script or prompt the user for a different directory

# Load the images
imgs = [Image.open(f) for f in img_files]

# Convert the list of PIL images to a NumPy array
imgs_array = np.array([np.array(img) for img in imgs])


Error: No '.jpg' files found in the directory.


In [ ]:
   # Definir a arquitetura do modelo
generator = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(100,)),
    tf.keras.layers.Dense(4 * 4 * 128, activation='relu'),  # Adjust the number of neurons in the Dense layer
    tf.keras.layers.Reshape((4, 4, 128)),
    tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.Activation('tanh')
])


discriminator = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(64, 64, 3)),
    tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
generator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.compile(loss='binary_crossentropy', optimizer='adam')



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:

# Wrap the generator model with KerasClassifier
generator_classifier = KerasClassifier(model=generator)

# Define the parameter grid for the generator
params = {
    'model__learning_rate': [0.001, 0.01, 0.1],  # Note the 'model__' prefix
    'batch_size': [32, 64, 128]
}

# You'll need to define y_train with labels for your data
# For demonstration, let's assume all images are of a single class (label 0)
# Ensure that y_train has the same number of samples as imgs_array
y_train = np.zeros(imgs_array.shape[0])

# Perform Grid Search on the wrapped generator
grid_search = GridSearchCV(generator_classifier, params, cv=5, scoring='accuracy')
grid_search.fit(imgs_array, y_train)

# Print the best parameters found
print(grid_search.best_params_)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.